In [8]:
# basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Image processing libraries
import cv2
import os
from PIL import Image
# DL/ML Libraries
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout

In [9]:
classes = 43
data = []
labels = []

metaDf = pd.read_csv('gtsrb-german-traffic-sign/Meta.csv')
trainDf = pd.read_csv('gtsrb-german-traffic-sign/Train.csv')
testDf = pd.read_csv('gtsrb-german-traffic-sign/Test.csv')

labels = ['20 km/h', '30 km/h', '50 km/h', '60 km/h', '70 km/h', '80 km/h', '80 km/h end', '100 km/h', '120 km/h', 'No overtaking',
               'No overtaking for tracks', 'Crossroad with secondary way', 'Main road', 'Give way', 'Stop', 'Road up', 'Road up for track', 'Brock',
               'Other dangerous', 'Turn left', 'Turn right', 'Winding road', 'Hollow road', 'Slippery road', 'Narrowing road', 'Roadwork', 'Traffic light',
               'Pedestrian', 'Children', 'Bike', 'Snow', 'Deer', 'End of the limits', 'Only right', 'Only left', 'Only straight', 'Only straight and right', 
               'Only straight and left', 'Take right', 'Take left', 'Circle crossroad', 'End of overtaking limit', 'End of overtaking limit for track']

In [10]:
print('SHAPE of training set:',trainDf.shape)
print('SHAPE of test set:',testDf.shape)
print('SHAPE of MetaInfo:',metaDf.shape)

SHAPE of training set: (39209, 8)
SHAPE of test set: (12630, 8)
SHAPE of MetaInfo: (43, 5)


In [11]:
# Loading the path
trainDf['Path'] = list(map(lambda x: os.path.join(data_dir,x.lower()),trainDf['Path']))
testDf['Path'] = list(map(lambda x: os.path.join(data_dir,x.lower()), testDf['Path']))
metaDf['Path'] = list(map(lambda x: os.path.join(data_dir,x.lower()), metaDf['Path']))

In [12]:
trainDf.sample(10)
testDf.sample(10)
metaDf.sample(10)

,Path,ClassId,ShapeId,ColorId,SignId
5,/kaggle/input/gtsrb-german-traffic-sign\meta/1...,12,2,2,2.3
7,/kaggle/input/gtsrb-german-traffic-sign\meta/1...,14,3,0,2.2
32,/kaggle/input/gtsrb-german-traffic-sign\meta/3...,38,1,1,4.7
27,/kaggle/input/gtsrb-german-traffic-sign\meta/3...,33,1,1,4.2
31,/kaggle/input/gtsrb-german-traffic-sign\meta/3...,37,1,1,4.5
0,/kaggle/input/gtsrb-german-traffic-sign\meta/2...,27,0,0,1.32
15,/kaggle/input/gtsrb-german-traffic-sign\meta/2...,21,0,0,1.3.2
13,/kaggle/input/gtsrb-german-traffic-sign\meta/2...,2,1,0,3.29
3,/kaggle/input/gtsrb-german-traffic-sign\meta/1...,10,1,0,3.27
34,/kaggle/input/gtsrb-german-traffic-sign\meta/4...,4,1,0,3.29


In [ ]:
# Analyzing the class distribution using histogram
fig,axs = plt.subplots(1,2,sharex=True, sharey=True,figsize=(25,6))
axs[0].set_title('train classes distribution')
axs[0].set_xlabel('Class')
axs[0].set_ylabel('Count')
axs[1].set_title('test classes distribution')
axs[1].set_xlabel('Class')
axs[1].set_ylabel('Count')

sns.countplot(x=trainDf.ClassId, hue=trainDf.ClassId, ax=axs[0], palette="Set1", legend=False)
sns.countplot(testDf.ClassId,ax=axs[1])
axs[0].set_xlabel('Class ID');
axs[1].set_xlabel('Class ID');

In [ ]:
# Visualize the Target class
sns.set_style()
rows=6
cols=8
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True,figsize=(25,12))
plt.subplots_adjust(left=None,bottom=None,right=None,top=0.9,wspace=None,hspace=None)
metaDf = metaDf.sort_values(by=['ClassId'])
idx=0
for i in range(rows):
    for j in range(cols):
        if idx>42:
            break

        img = cv2.imread(metaDf["Path"].tolist()[idx],cv2.IMREAD_UNCHANGED)
        img[np.where(img[:,:,3]==0)] = [255,255,255,255]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (60,60))
        
        axs[i,j].imshow(img)
        axs[i,j].set_facecolor('xkcd:salmon')
        axs[i,j].set_facecolor((1.0, 0.47, 0.42))
        axs[i,j].set_title(labels[int(metaDf["ClassId"].tolist()[idx])])
        axs[i,j].get_xaxis().set_visible(False)
        axs[i,j].get_yaxis().set_visible(False)
        idx += 1

In [ ]:
# Visualizing the Training set
rows = 10
cols = 10
fig, axs = plt.subplots(rows, cols, sharex=True, sharey=True, figsize=(25, 12))
plt.subplots_adjust(left=None, bottom=None, right=None, top=0.9, wspace=None, hspace=None)
cur_path = '/kaggle/input/gtsrb-german-traffic-sign'
print(cur_path)
idx = 0
for i in range(rows):
    for j in range(cols):
        path = os.path.join(cur_path,trainDf["Path"].tolist()[idx]) 
        img = cv2.imread(path,cv2.IMREAD_UNCHANGED)
        #print(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (60,60))
        axs[i,j].imshow(img)
        axs[i,j].set_title(labels[int(trainDf["ClassId"].tolist()[idx])])
        axs[i,j].get_xaxis().set_visible(False)
        axs[i,j].get_yaxis().set_visible(False)
        idx += 1

In [ ]:
# extracting the path for training,testing,label
data=[]
labels=[]
classes=43
cur_path = os.getcwd()
print(cur_path)

In [ ]:
# Retrieving the images and their labels
for i in range(classes):
    path = f'/kaggle/input/gtsrb-german-traffic-sign/Train/{i}/'

    try:
        images = os.listdir(path)
    except:
        print(f"Error reading directory: {path}")
        continue

    print(f"Loading images from: {path}")

    # Coverting lists into numoy arrays
    for a in images:
        try:
            image = Image.open(os.path.join(path, a))
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [ ]:
# read the images and covert into numpy arrays
data = np.array(data)
labels = np.array(labels)
print(data.shape,labels.shape)

In [ ]:
# splitting the trwaining set
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# One hot encoding on labels
y_train = to_categorical(y_train,43)
y_test = to_categorical(y_test,43)

In [ ]:
## Building the CNN model
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),activation='relu',input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32,kernel_size=(5,5),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(43,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
epochs = 15
history = model.fit(X_train,y_train,batch_size=64,epochs=epochs,validation_data=(X_test,y_test))

In [ ]:
# plotting the graphs for accuracy
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#testing accuracy on test dataset
cur_path = os.getcwd()
print(cur_path)
#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'/kaggle/input/gtsrb-german-traffic-sign/Test')
    images = os.listdir(path)
    # print(path)
    # print(images)

In [ ]:
from sklearn.metrics import accuracy_score

# Flatten labels if necessary
labels = np.array(labels)
if labels.ndim > 1:
    labels = labels.ravel()

# Convert predictions to class labels
pred_labels = np.argmax(pred, axis=1)

print("Accuracy:", accuracy_score(labels, pred_labels))